<a href="https://colab.research.google.com/github/LissetteAlvarez/VectorialDB/blob/main/pineconedemo_vectordatabases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pinecone demo básica (simple pinecone demo)

La librería pinecone-client nos permitirá acceder al servicio de la base de datos Pinecone.

Utilizaremos la librería sentence-transformer para generar los embeddings de texto que añadiremos a la base de datos.

Instalemos las librerias:


In [ ]:
pip install pinecone-client

In [ ]:
pip install sentence-transformers

##  Generando embeddings
En primer lugar codificaremos un conjunto de datos de texto (**embeddings**) para almacenarlos en la base de datos vectorial. Para ello, utilizaremos la biblioteca SentenceTransformers.

Esta biblioteca proporciona arquitecturas basadas en **`Transformers`** pre-entrenados que pueden codificar texto de forma eficiente en representaciones vectoriales densas (embeddings). Los embeddings capturan similitudes semánticas y relaciones entre textos, lo que las hace adecuadas para tareas posteriores de clasificación y agrupación.

**`SentenceTransformers`** ofrece varias arquitecturas pre-entrenadas como BERT, RoBERTa y DistilBERT. En esta demo utilizaremos DistilBERT, ya que  es un modelo relativamente ligero.

In [ ]:
from pinecone import Pinecone, PodSpec
from sentence_transformers import SentenceTransformer

Creamos una instancia de DistilBERT:

In [ ]:
# Modelo
model_name = 'distilbert-base-nli-stsb-mean-tokens'
model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Para acceder al servicio de Pinecone, necesitamos una API key.
Puedes encontrarla accediendo a tu cuenta Pinecone y accediendo a la sección:  
`API Keys -> Create API Key -> Enter API Key Name -> Create`.

Si aún no posees cuenta en Pinecone, puedes crearla aquí:
https://app.pinecone.io/?sessionType=signup.


Hay dos formas de añadir la API Key si estás codificando en Google Colab: añadiendo directamente la contraseña al código o gestionando las variables de entorno almacenadas en 'Secretos'.

Veamos ambos métodos.



## Conectar Pinecone desde el código
Añade la API key al código:

In [ ]:
# Pinecone connect
# pinecone_key = "<Add touy Pinecone API key here>"
# pc_service = Pinecone(api_key = pinecone_key)

## Gestionando las variables de entorno de Google Colab:

Para la API key desde el servicio de gestión de variables de entorno de Google Colab (**Secrets**), recuperamos la API key con `userdata.get()`. En mi caso, la he almacenado con el nombre 'Pinecone_key'.

Es importante dar acceso al cuaderno a la variable de entorno correspondiente, activando el botón 'Acces desde el cuaderno' de la sección '**Secretos**' disponible en el panel lateral izquierdo:

![](https://drive.google.com/uc?export=view&id=1g4FDpxdPqQbQZIovOkubLtRBshwli0Ji)



In [ ]:
from google.colab import userdata
import os
pinecone_key = userdata.get('Pinecone_key')
os.environ['pinecone_key'] = pinecone_key
pc_service = Pinecone(api_key = pinecone_key)

## Generando índices y embeddings

En Pinecone, los embeddings se almacenan en índices. Todos los embeddigns deben calcularse con la misma distancia y tener la misma dimensionalidad para medir adecuadamente la similitud entre ellos.

En primer lugar utilizamos el `método list_indexes()` para crear una lista vacía de índices. Luego, creamos los índices utilizando el método `create_index()`.



In [ ]:
 # Lista vacía de índices
 pc_service.list_indexes()

{'indexes': []}

In [ ]:
# Crear índices
pc_service.create_index(
    name="vector-demo",
    dimension=768,
    metric="euclidean",
    spec=PodSpec(environment="gcp-starter")
  )

Los argumentos de la función `create_index()` son:

**`name`**: nombre del índice (definido por el usuario). Puede utilizarse para referirse al índice más adelante cuando se realicen operaciones sobre él.

**`dimension`**: define la dimensionalidad de los vectores que se almacenarán en el índice. Debe coincidir con la dimensionalidad de los vectores que se insertarán en el índice. En este caso, el valor 768 corresponde a la dimensión de embedding devuelta por el modelo `SentenceTransformer`.

**`metric`**: métrica de distancia utilizada para calcular la similitud entre vectores.

**`spec`**: Un objeto PodSpec que especifica el entorno en el que se creará el índice. En este ejemplo, el índice se crea en un entorno GCP (Google Cloud Platform) llamado `gcp-starter`.

Una vez creados los índices, agregamos los vectores de embeddings.

Para ello, creamos un dataset de texto como ejemplo y lo codificamos utilizando el modelo `SentenceTransformer` model:

In [ ]:
# dataset de ejemplo
data = [
    {"id": "vector1",  "text": "I love using vector databases"},
    {"id": "vector2",  "text": "Vector databases are great for storing and retrieving vectors"},
    {"id": "vector3",  "text": "Using vector databases makes my life easier"},
    {"id": "vector4",  "text": "Vector databases are efficient for storing vectors"},
    {"id": "vector5",  "text": "I enjoy working with vector databases"},
    {"id": "vector6",  "text": "Vector databases are useful for many applications"},
    {"id": "vector7",  "text": "I find vector databases very helpful"},
    {"id": "vector8",  "text": "Vector databases can handle large amounts of data"},
    {"id": "vector9",  "text": "I think vector databases are the future of data storage"},
    {"id": "vector10", "text": "Using vector databases has improved my workflow"}
]

In [ ]:
# Embeddings
vector_data = []

for sentence in data:

    embedding = model.encode(sentence["text"])
    vector_info = {"id":sentence["id"], "values": embedding.tolist()}

    vector_data.append(vector_info)

Este fragmento de código codifica el texto de cada frase en un vector utilizando el modelo `sentence transformer`.

A continuación, crea un diccionario (`vector_info`) que contiene el ID de la frase (`id`) y el vector correspondiente (`values`), y añade este diccionario a la lista `vector_data`.

Cuando existen múltiples índices bajo la misma cuenta, debemos crear un objeto índice que indique a cuál índice deseamos añadir los embeddings:

In [ ]:
# Especificar índice para los embeddings
index = pc_service.Index("vector-demo")

## Insertando los embeddings a la DB

Una vez generados los embeddings y el índice donde deseamos agregarlos, podemos insertar los embeddings. Este proceso, conocido como 'upsert', combina las acciones de actualización e inserción:

*   Inserta un nuevo documento en una colección si el documento aún no existe, o
*   Actualiza un documento existente si ya existe.


In [ ]:
# Upsert embeddings to index
index = pc_service.Index("vector-demo")

index.upsert(vectors=vector_data)

{'upserted_count': 10}

Listo! El proceso a añadido (o actualzado) 10 registros.

Ppodemos utilizar `describe_index_stats` para comprobar que la cantidad de vectores actuales coincide con el número de vectores que hemos añadido:

In [ ]:
# Comprobación
index.describe_index_stats()


{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

La salida nos ofrece la siguiente información:


* **`dimensión`**: La dimensionalidad de los vectores almacenados en el índice (768, en este caso)
*   **`index_fullness`**: indica el porcentaje de espacios en el índice que están ocupados
*   **`namespaces`**: diccionario que contiene estadísticas para cada namespaces en el índice. En este caso, solo hay un namespace (`''`) con un `vector_count` de 10, lo que indica que hay 10 vectores en el índice.
*   **`total_vector_count`**: número total de vectores en el índice en todos los namespaces (10, en este caso).




## Búsqueda en Pinecone
Ahora que tenemos una base de datos en Pinecone, ejecutemos una búsqueda de con base en la similitud de los vectores.

Primero, definimos un texto de búsqueda, esto es, un nuevo texto que tendremos que codificar para luego buscar el o los vectores con mayor similitud dentro de la base de datos.





In [ ]:
# Texto de búsqueda
search_text = "Vector database are really helpful"

# embedding
search_embedding = model.encode(search_text).tolist()

Utilizamos el método `query()` para identificar los 3 'vecinos más cercanos':

In [ ]:
# Vectores más cercanos neighbors
index.query(vector=search_embedding, top_k=3)

{'matches': [], 'namespace': '', 'usage': {'read_units': 5}}

In [ ]:
iq = index.query(vector=search_embedding, top_k=3, include_metadata=True)
for resultado in iq['matches']:
    print(resultado['id'] )

In [ ]:
# closest neighbors
print(data[7]['text'])
print(data[4]['text'])
print(data[5]['text'])

Vector databases can handle large amounts of data
I enjoy working with vector databases
Vector databases are useful for many applications


## Eliminamos el índice en Pinecone


In [ ]:
pc_service.delete_index("vector-demo")